# Solve problem 441618 (850083 nodes) with pyAMG to get a baseline timing comparison

The corresponding inference against the GNN on the 48-core graphcore machine of this size took 5mins and 391GB RAM


In [2]:
!pip3 install --user google-cloud-storage pyamg

     |████████████████████████████████| 112kB 5.2MB/s eta 0:00:01
     |████████████████████████████████| 153kB 8.7MB/s eta 0:00:01
     |████████████████████████████████| 61kB 5.8MB/s eta 0:00:011
     |████████████████████████████████| 81kB 5.8MB/s eta 0:00:011
     |████████████████████████████████| 163kB 7.8MB/s eta 0:00:01
     |████████████████████████████████| 153kB 8.3MB/s eta 0:00:01
     |████████████████████████████████| 61kB 6.0MB/s  eta 0:00:01
     |████████████████████████████████| 184kB 8.6MB/s eta 0:00:01
     |████████████████████████████████| 102kB 6.1MB/s ta 0:00:011
     |████████████████████████████████| 81kB 6.0MB/s  eta 0:00:01
     |████████████████████████████████| 1.0MB 9.1MB/s eta 0:00:01
     |████████████████████████████████| 512kB 17.9MB/s eta 0:00:01
     |████████████████████████████████| 40kB 6.8MB/s  eta 0:00:01
     |████████████████████████████████| 204kB 31.5MB/s eta 0:00:01
     |████████████████████████████████| 409kB 24.6MB/s eta 0:00:01
     |█

In [10]:
from google.cloud import storage
import os
import concurrent
from pathlib import Path
import itertools
from scipy.sparse import csr_matrix, load_npz
from typing import Tuple
import numpy as np
import time
import pyamg


In [11]:

GOOGLE_APPLICATION_CREDENTIALS="/home/fenics/shared/bucket-key.json"
BUCKET_NAME = 'learning-amg-ftetwild-matrices'
storage_client = storage.Client.from_service_account_json(GOOGLE_APPLICATION_CREDENTIALS)
bucket = storage_client.get_bucket(BUCKET_NAME)
blobs = storage_client.list_blobs('learning-amg-ftetwild-matrices')

def download_blobs_for_problem_id(problem_id: str) -> bool:
    blobs = storage_client.list_blobs(BUCKET_NAME, prefix=problem_id)
    a = None
    b = None
    with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
        futures = []
        for blob in blobs:
            if blob.name.endswith("velocity_A.npz"):
                a = blob.name
                futures.append(executor.submit(blob.download_to_filename, "A.npz"))
            if blob.name.endswith("velocity_b.npz"):
                b = blob.name
                futures.append(executor.submit(blob.download_to_filename, "b.npz"))
            if a is not None and b is not None:
                break
        concurrent.futures.wait(futures, return_when=concurrent.futures.ALL_COMPLETED)
    return a is not None and b is not None

def load_A_and_b(problem_id: str) ->  Tuple[csr_matrix, ]:
    download_ok = download_blobs_for_problem_id(problem_id)
    if (download_ok):
        a = load_npz("A.npz")
        b = load_npz("b.npz")
    else:
        raise Exception("Could not download A and b for problem {}".format(problem_id))
    return a, b.todense().flatten().T






In [12]:
A, b = load_A_and_b('441618')

In [13]:

accel = None
maxiter = 500
tol=1e-12
CF='CLJP'
coarse_method='pinv2'
strength=('classical', {'theta': 0.25})
strength='algebraic_distance'
presmoother=('gauss_seidel', {'sweep': 'symmetric', 'iterations': 1})
postsmoother=('gauss_seidel', {'sweep': 'symmetric', 'iterations': 1})
max_levels=12
cycle='V'

        
residuals = []
#ml = pyamg.smoothed_aggregation_solver(A, max_coarse=10)
#x = ml.solve(b, tol=1e-12, maxiter=40, accel='cg', residuals=residuals)
tic = time.time()
ml = pyamg.classical.ruge_stuben_solver(A, 
#                                             max_coarse=max_coarse, 
                                        strength=strength,
                                        presmoother=presmoother,
                                        postsmoother=postsmoother,
                                        CF=CF,
                                        coarse_solver=coarse_method)
residuals = []
tic = time.time()
x = ml.solve(b, tol=tol, maxiter=maxiter, residuals=residuals, accel=accel, cycle=cycle) 
toc = time.time()
msg = repr(ml)
residuals = np.array(residuals)
result = residuals[-1] < 1e-10
residuals = residuals[residuals >= tol]


SyntaxError: 'return' outside function (<ipython-input-13-831346e03375>, line 33)

In [ ]:
timing = toc - tic
num_iter = len(residuals)
x = result

messages = []
messages.append("Problem {}".format(problem))
messages.append("Size of A: {}".format(A.shape[0]))
messages.append("Took: {:.5f}s".format(toc-tic))
messages.append("Num iterations: {}".format(num_iter))
messages.append("Could solve?: {}".format(x))
messages.append("Residuals: {}".format(residuals))
messages.append("Details: {}".format(descr))
       
for m in messages:
    print(m)